In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import findspark
findspark.init()

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.functions import min

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [2]:
data_rdd = spark.read.csv('en.openfoodfacts.org.products.csv',header=True,sep='\t') 
                
               

In [9]:
data_rdd.columns

['code',
 'url',
 'creator',
 'created_t',
 'created_datetime',
 'last_modified_t',
 'last_modified_datetime',
 'product_name',
 'generic_name',
 'quantity',
 'packaging',
 'packaging_tags',
 'brands',
 'brands_tags',
 'categories',
 'categories_tags',
 'categories_en',
 'origins',
 'origins_tags',
 'manufacturing_places',
 'manufacturing_places_tags',
 'labels',
 'labels_tags',
 'labels_en',
 'emb_codes',
 'emb_codes_tags',
 'first_packaging_code_geo',
 'cities',
 'cities_tags',
 'purchase_places',
 'stores',
 'countries',
 'countries_tags',
 'countries_en',
 'ingredients_text',
 'allergens',
 'allergens_en',
 'traces',
 'traces_tags',
 'traces_en',
 'serving_size',
 'serving_quantity',
 'no_nutriments',
 'additives_n',
 'additives',
 'additives_tags',
 'additives_en',
 'ingredients_from_palm_oil_n',
 'ingredients_from_palm_oil',
 'ingredients_from_palm_oil_tags',
 'ingredients_that_may_be_from_palm_oil_n',
 'ingredients_that_may_be_from_palm_oil',
 'ingredients_that_may_be_from_palm_

In [19]:
#analyse and cleaning the data, choosing what to do according what data we have: FIRST WEEK

data_rdd.groupBy("packaging").count().orderBy('count',ascending=False).show(5)
from pyspark.sql import functions as F
total = packaging.groupBy().agg(F.sum("count")).collect()
#the first analys on packaging we meant to be , it can not take place because the amount of null is way higher respect to the total

+----------------+------+
|       packaging| count|
+----------------+------+
|            null|553446|
|sachet,plastique|  4425|
|Sachet,Plastique|  3160|
| Bouteille,Verre|  2696|
|       Plastique|  2491|
+----------------+------+
only showing top 5 rows



In [25]:
import re
data_rdd.groupBy('brands').count().orderBy('count',ascending=False).show(25)

+---------------+------+
|         brands| count|
+---------------+------+
|           null|229610|
|      Carrefour|  5410|
|         Auchan|  5324|
|              U|  4405|
|         Casino|  3129|
|   Leader Price|  2846|
|           Cora|  2234|
|         Meijer|  1997|
|         Kroger|  1673|
|         Picard|  1521|
|          Ahold|  1370|
|          Netto|  1349|
|        Spartan|  1341|
|         Nestlé|  1322|
|    Great Value|  1304|
|       Roundy's|  1299|
|       Monoprix|  1245|
|       Delhaize|  1190|
|           Coop|  1132|
|   Belle France|  1130|
|           Weis|  1055|
|  La Vie Claire|   997|
|Marks & Spencer|   995|
|       Franprix|   971|
|  Fleury Michon|   962|
+---------------+------+
only showing top 25 rows



In [24]:
data_rdd
data_rdd.groupBy(["brands","product_name"]).count().orderBy('count',ascending=False).where(data_rdd.brands.isNotNull()).filter(lambda x:x=='Carrefour').show(10)

TypeError: condition should be string or Column

In [37]:
data_rdd.groupBy("categories_tags").count().orderBy('count',ascending=False).show(5)

+--------------------+------+
|     categories_tags| count|
+--------------------+------+
|                null|514309|
|en:beverages,en:n...|  5124|
|             en:fats|  1799|
|en:beverages,en:s...|  1674|
|en:sugary-snacks,...|  1368|
+--------------------+------+
only showing top 5 rows



In [17]:
data_rdd.groupBy("brands").count().orderBy('count',ascending=False).show(5)

AnalysisException: "cannot resolve '`Carrefour`' given input columns: [brands, count]; line 1 pos 0;\n'Filter 'Carrefour\n+- AnalysisBarrier\n      +- Sort [count#1291L DESC NULLS LAST], true\n         +- Aggregate [brands#22], [brands#22, count(1) AS count#1291L]\n            +- Relation[code#10,url#11,creator#12,created_t#13,created_datetime#14,last_modified_t#15,last_modified_datetime#16,product_name#17,generic_name#18,quantity#19,packaging#20,packaging_tags#21,brands#22,brands_tags#23,categories#24,categories_tags#25,categories_en#26,origins#27,origins_tags#28,manufacturing_places#29,manufacturing_places_tags#30,labels#31,labels_tags#32,labels_en#33,... 149 more fields] csv\n"

In [40]:
data_rdd.groupBy("origins").count().orderBy('count',ascending=False).show(5)#france,France,FR

+----------------+------+
|         origins| count|
+----------------+------+
|            null|650941|
|          France| 11016|
|Union Européenne|  1524|
|          Italie|   936|
|         Espagne|   865|
+----------------+------+
only showing top 5 rows



In [42]:
data_rdd.groupBy("manufacturing_places").count().orderBy('count',ascending=False).show(5)#france,France,FR

+--------------------+------+
|manufacturing_places| count|
+--------------------+------+
|                null|626138|
|              France| 18422|
|              Italie|  2370|
|            Belgique|  1408|
|           Allemagne|  1241|
+--------------------+------+
only showing top 5 rows



In [44]:
data_rdd.groupBy("labels").count().orderBy('count',ascending=False).show(5)

+--------------------+------+
|              labels| count|
+--------------------+------+
|                null|591148|
|          en:organic| 13371|
|          Point Vert|  5234|
|                 Bio|  2593|
|Bio,Bio européen,...|  1906|
+--------------------+------+
only showing top 5 rows



In [46]:
data_rdd.groupBy("purchase_places").count().orderBy('count',ascending=False).show(5)

+-----------------+------+
|  purchase_places| count|
+-----------------+------+
|             null|592657|
|           France| 30886|
|      Lyon,France|  3464|
|Courrières,France|  2474|
|    France,Nantes|  2430|
+-----------------+------+
only showing top 5 rows



In [47]:
data_rdd.groupBy("stores").count().orderBy('count',ascending=False).show(5)

+----------+------+
|    stores| count|
+----------+------+
|      null|602239|
|Magasins U| 11800|
| Carrefour|  7098|
|   Leclerc|  4288|
|    Auchan|  3981|
+----------+------+
only showing top 5 rows



In [3]:
data_rdd.groupBy("countries_en").count().orderBy('count',ascending=False).show(5)

+-------------+------+
| countries_en| count|
+-------------+------+
|       France|418539|
|United States|173557|
|  Switzerland| 13488|
|      Germany| 11807|
|        Spain|  6169|
+-------------+------+
only showing top 5 rows



In [53]:
data_rdd.groupBy("saturated-fat_100g").count().orderBy('count',ascending=False).show(5)

+------------------+------+
|saturated-fat_100g| count|
+------------------+------+
|              null|133117|
|                 0|124448|
|               0.1| 25871|
|               0.5| 10980|
|               0.2| 10751|
+------------------+------+
only showing top 5 rows



In [55]:
#for all the column count the numbe rof the null values:
data_rdd.filter(lambda x: sum(isnull(x)))


TypeError: condition should be string or Column